# Установка некоторых внешних зависимостей

In [21]:
!pip install -qU https://github.com/kpu/kenlm/archive/master.zip sacremoses

# Подключение библиотек

In [23]:
import gzip

import kenlm
from sacremoses import MosesDetokenizer

# MOSES

## Подготовка данных для обучения переводчика

Скачать данные в папку `/mnt/DATA/MSU2024NLP/corpus` и распаковать ее там (должны получить папку training)

In [ ]:
# !wget http://www.statmt.org/wmt13/training-parallel-nc-v8.tgz
# !tar zxvf training-parallel-nc-v8.tgz

### Токенизация текста

В данном случае мы используем токенизатор, который представлен разработчиками moses. На текущий момент он все еще обновляется (добаляются новые языки и тд).

Всегда можно использовать другие реализации токенизаторов (например соверменные BPE/WordPiece). В целом используя такого рода токенизаторы возможно получить прирост качества на некоторых языках, а также получить мультиязычный статистический перевод.

[1] Asvarov A., Grabovoy A. The impact of multilinguality and tokenization on statistical machine translation // 2024 35th Conference of Open Innovations Association (FRUCT). — IEEE: 2024. — P. 149–157.

In [1]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/tokenizer/tokenizer.perl -l en \
        < /corpus/training/news-commentary-v8.ru-en.en \
        > /corpus/news-commentary-v8.ru-en.tok.en"

Tokenizer Version 1.1
Language: en
Number of threads: 1


In [3]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/tokenizer/tokenizer.perl -l ru \
        < /corpus/training/news-commentary-v8.ru-en.ru > \
        /corpus/news-commentary-v8.ru-en.tok.ru"

Tokenizer Version 1.1
Language: ru
Number of threads: 1


Получили выборку размером в 150тыс предложений для обучения статистического переводчика. Выборка из примера, имеет целый спектр недостатков (и в целом достаточно маленькая для статистического переводочика).

Заметка из опыта:
1. Для обучения неплохого статистического переводчика требуется несколько милионов пар предложений.
2. Параллельные предложения для разных языков можно брать из https://opus.nlpl.eu.
3. На качество перевода очень сильно влияет качество данных и качество токенизатора.

In [3]:
!wc -l /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.tok.en

150217 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.tok.en


In [4]:
!wc -l /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.tok.ru

150217 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.tok.ru


In [4]:
!head -n 10 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.tok.en

SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold .
Lately , with gold prices up more than 300 % over the last decade , it is harder than ever . Just last December , fellow economists Martin Feldstein and Nouriel Roubini each penned op-eds bravely questioning bullish market sentiment , sensibly pointing out gold ’ s risks .
Wouldn ’ t you know it ?
Since their articles appeared , the price of gold has moved up still further .
Gold prices even hit a record-high $ 1,300 recently .
Last December , many gold bugs were arguing that the price was inevitably headed for $ 2,000 .
Now , emboldened by continuing appreciation , some are suggesting that gold could be headed even higher than that .
One successful gold investor recently explained to me that stock prices languished for a more than a decade before the Dow Jones index crossed the 1,000 mark in the early 1980 ’ s .
Since then , the index has climbed above 10,000 .
Now that gold has crossed th

In [5]:
!head -n 10 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.tok.ru

САН-ФРАНЦІСКО . Разговор о стоимости золота редко получается рациональным , тем более в последнее время , так как цены на золото выросли более чем на 300 % за десятилетие .
Еще в декабре прошлого года экономисты-коллеги Мартин Фельдштейн и Нуриэль Рубини опубликовали свои пророческие статьи в колонках альтернативных мнений , храбро ставя в них под вопрос стремление игры на повышение , благоразумно указывая на риски золота .
И что бы вы думали ?
С тех пор как вышли их статьи , стоимость золота повысилась еще больше .
Недавно цена на золото даже достигла рекордной отметки в 1 300 долларов за унцию .
В декабре прошлого года многие сторонники сохранения денежных функций золота утверждали , что его цена неизбежно дойдет до 2 000 долларов .
Теперь , воодушевленные постоянным ростом цен , некоторые считают , что цена на золото может вырасти еще сильнее .
Один успешный инвестор в золото недавно объяснил мне , что цена акций падала в течение более десятилетия , прежде чем в начале 1980-х гг. ин

### Востановления регистра первого слова предложения

В целом регистры слов (с большой буквы или маленькой) очень сильно влияют на различные модели NLP. Часто строятся модели, которые работают только с нижним регистров (весь текст переводят в lower). В целом и тут можно было бы сделать так, но в документации рекомендуют сделать "перераспределение" регистров слов, чтобы на первое слово в предложении не влиял регистр, а он влиял только на те слова, которые чаще встречаются в верхнем регистре.

#### Обучение

In [5]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/recaser/train-truecaser.perl \
        --model /corpus/truecase-model.en \
        --corpus /corpus/news-commentary-v8.ru-en.tok.en"

In [6]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/recaser/train-truecaser.perl \
        --model /corpus/truecase-model.ru \
        --corpus /corpus/news-commentary-v8.ru-en.tok.ru"

#### Применение

In [7]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/recaser/truecase.perl \
        --model /corpus/truecase-model.en \
        < /corpus/news-commentary-v8.ru-en.tok.en \
        > /corpus/news-commentary-v8.ru-en.true.en"

In [9]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/recaser/truecase.perl \
        --model /corpus/truecase-model.ru \
        < /corpus/news-commentary-v8.ru-en.tok.ru \
        > /corpus/news-commentary-v8.ru-en.true.ru"

In [8]:
!wc -l /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.true.en

150217 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.true.en


In [10]:
!wc -l /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.true.ru

150217 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.true.ru


In [6]:
!head -n 10 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.true.en

San FRANCISCO – It has never been easy to have a rational conversation about the value of gold .
lately , with gold prices up more than 300 % over the last decade , it is harder than ever . just last December , fellow economists Martin Feldstein and Nouriel Roubini each penned op-eds bravely questioning bullish market sentiment , sensibly pointing out gold ’ s risks .
wouldn ’ t you know it ?
since their articles appeared , the price of gold has moved up still further .
gold prices even hit a record-high $ 1,300 recently .
last December , many gold bugs were arguing that the price was inevitably headed for $ 2,000 .
now , emboldened by continuing appreciation , some are suggesting that gold could be headed even higher than that .
one successful gold investor recently explained to me that stock prices languished for a more than a decade before the Dow Jones index crossed the 1,000 mark in the early 1980 ’ s .
since then , the index has climbed above 10,000 .
now that gold has crossed th

In [7]:
!head -n 10 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.true.ru

САН-ФРАНЦІСКО . разговор о стоимости золота редко получается рациональным , тем более в последнее время , так как цены на золото выросли более чем на 300 % за десятилетие .
еще в декабре прошлого года экономисты-коллеги Мартин Фельдштейн и Нуриэль Рубини опубликовали свои пророческие статьи в колонках альтернативных мнений , храбро ставя в них под вопрос стремление игры на повышение , благоразумно указывая на риски золота .
и что бы вы думали ?
с тех пор как вышли их статьи , стоимость золота повысилась еще больше .
недавно цена на золото даже достигла рекордной отметки в 1 300 долларов за унцию .
в декабре прошлого года многие сторонники сохранения денежных функций золота утверждали , что его цена неизбежно дойдет до 2 000 долларов .
теперь , воодушевленные постоянным ростом цен , некоторые считают , что цена на золото может вырасти еще сильнее .
один успешный инвестор в золото недавно объяснил мне , что цена акций падала в течение более десятилетия , прежде чем в начале 1980-х гг. ин

В целом после предобработки данных видно, что слова теперь не всегда начинаются с верхнего регистра, а только в тех случаях, когда это обосновано.

### Фильтрация предложений по длине (1-80символов)

In [11]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/training/clean-corpus-n.perl \
        /corpus/news-commentary-v8.ru-en.true ru en \
        /corpus/news-commentary-v8.ru-en.clean 1 80"

clean-corpus.perl: processing /corpus/news-commentary-v8.ru-en.true.ru & .en to /corpus/news-commentary-v8.ru-en.clean, cutoff 1-80, ratio 9
..........(100000).....
Input sentences: 150217  Output sentences:  146806


In [12]:
!wc -l /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.clean.en

146806 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.clean.en


In [13]:
!wc -l /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.clean.ru

146806 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.clean.ru


## Обучение языковой модели

Языковая модель (LM) используется для обеспечения корректного написания перевода, поэтому она построена на целевом языке. В примере строится 3-граммная языковая модель (на базе библиотеки KenLM).

In [15]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/bin/lmplz \
        -o 3 \
        < /corpus/news-commentary-v8.ru-en.true.en \
        > /corpus/news-commentary-v8.ru-en.arpa.en"

=== 1/5 Counting and sorting n-grams ===
Reading /corpus/news-commentary-v8.ru-en.true.en
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 4008930 types 62481
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:749772 2:4672178176 3:8760334336
Statistics:
1 62481 D1=0.624963 D2=0.967568 D3+=1.32128
2 905932 D1=0.742376 D2=1.08513 D3+=1.36658
3 2382985 D1=0.837421 D2=1.16432 D3+=1.35272
Memory estimate for binary LM:
type    MB
probing 63 assuming -p 1.5
probing 68 assuming -r models -p 1.5
trie    25 without quantization
trie    14 assuming -q 8 -b 8 quantization 
trie    24 assuming -a 22 array pointer compression
trie    12 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:749772 2:14494912 3:47659700
=== 4/5 Calcu

In [53]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/bin/build_binary \
        /corpus/news-commentary-v8.ru-en.arpa.en \
        /corpus/news-commentary-v8.ru-en.blm.en"

Reading /corpus/news-commentary-v8.ru-en.arpa.en
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


### ARPA формат языковой модели

В рамках данного формата подсчитаны все условные и полные вероятности по токенам в обучающей выборке. В формате:
1. Указываются все n-gram, которые запрашивались для обучения (в нашем случае 1/2/3-gram).
2. Указывается число n-gram, которое получилось после обучения (для каждого из 1/2/3).
3. Далее для каждой n-gram выдается:
    - Вероятность последнего токена в n-gram при условии предыдущих (если нет, то при условии токена \<s> --- начала): $p(w_{n}|w_1, w_2, \cdots w_{n-1})$.
    - Сама n-gram: $w_1, w_2, \cdots w_n$.
    - Полная вероятность встретить n-gram: $p(w_1, w_2, \cdots w_n)$

In [49]:
!head -n 20 /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.arpa.en

\data\
ngram 1=62481
ngram 2=905932
ngram 3=2382985

\1-grams:
-5.970607	<unk>	0
0	<s>	-1.3166543
-3.831133	</s>	0
-4.735731	San	-0.38199094
-5.8285656	FRANCISCO	-0.12937579
-2.2254546	–	-0.7769198
-4.735731	It	-0.27971995
-2.4997213	has	-0.7978452
-3.5793376	never	-0.29212165
-3.6418643	been	-0.2847993
-4.0782976	easy	-0.40742075
-2.0642972	to	-0.90560645
-2.6063344	have	-0.70169735
-2.3986692	a	-0.69319177


### Работа с языковой моделью при помощи KenLM

Языковые модели могут быть полезны в различных задачах NLP.

Пример: использование перплексии для детекции машинной генерации.

In [54]:
language_model = kenlm.Model('/mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.arpa.en')

sentence = 'What about your name ?'
language_model.score(sentence), language_model.perplexity(sentence)

Loading the LM will be faster if you build a binary file.
Reading /mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.arpa.en
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


(-16.963727951049805, 671.8742423267146)

In [55]:
language_model = kenlm.Model('/mnt/DATA/MSU2024NLP/corpus/news-commentary-v8.ru-en.blm.en')

sentence = 'What about your name ?'
language_model.score(sentence), language_model.perplexity(sentence)

(-16.963727951049805, 671.8742423267146)

## Обучение переводчика

Для этого производится:
- выравнивание слов (с помощью GIZA++);
- извлечение и оценку фраз;
- создание лексической таблицы реранжирования.

P.S. следующая ячейка работает порядка одного часа.

In [62]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/training/train-model.perl \
        -root-dir /corpus/train \
        -corpus /corpus/news-commentary-v8.ru-en.clean \
        -f ru -e en \
        -alignment grow-diag-final-and -reordering msd-bidirectional-fe \
        -lm 0:3:/corpus/news-commentary-v8.ru-en.blm.en \
        -external-bin-dir /opt/moses_tools/ \
        -cores 4 \
        --parallel -mgiza -mgiza-cpus 4 &> /corpus/training.log"

После обучения модели, получаем следующий конфигурационный файл, который полностью описывает модель переводчика.

Модель состоит из:
1. Таблицы вероятностей различных n-gram на языке источника и языке таргета.
2. Таблица перестановки слов:
    - wbe --- извлечение вероятнотней на базе отдельных слов внутри фраз;
    - msd --- последовательность перестановки слов в фразе;
    - bidirectional --- выбор как нужно вероятности считать слева на право или справа на лево (тут и те и те);
    - fe --- используем какие фразы, только исходного языка или и выходного тоже (тут оба);
    - allff --- строим много признаков для оценки вероятности.
3. Языкова модель выходного текста.

In [71]:
!cat /mnt/DATA/MSU2024NLP/corpus/train/model/moses.ini

#########################
### MOSES CONFIG FILE ###
#########################

# input factors
[input-factors]
0

# mapping steps
[mapping]
0 T 0

[distortion-limit]
6

# feature functions
[feature]
UnknownWordPenalty
WordPenalty
PhrasePenalty
PhraseDictionaryMemory name=TranslationModel0 num-features=4 path=/corpus/train/model/phrase-table.gz input-factor=0 output-factor=0
LexicalReordering name=LexicalReordering0 num-features=6 type=wbe-msd-bidirectional-fe-allff input-factor=0 output-factor=0 path=/corpus/train/model/reordering-table.wbe-msd-bidirectional-fe.gz
Distortion
KENLM name=LM0 factor=0 path=/corpus/news-commentary-v8.ru-en.blm.en order=3

# dense weights for feature functions
[weight]
# The default weights are NOT optimized for translation quality. You MUST tune the weights.
# Documentation for tuning is here: http://www.statmt.org/moses/?n=FactoredTraining.Tuning 
UnknownWordPenalty0= 1
WordPenalty0= -1
PhrasePenalty0= 0.2
TranslationModel0= 0.2 0.2 0.2 0.2
LexicalReorder

In [15]:
extracted = ''
with gzip.open('/mnt/DATA/MSU2024NLP/corpus/train/model/phrase-table.gz','rt') as f:
    for _ in range(1000):
        extracted += next(f)
print('\n'.join(extracted.split('\n')[150:200]))

! ) и регулирующим ||| ! ) and in ||| 1 7.22626e-06 1 0.0174206 ||| 0-0 1-1 2-2 3-3 ||| 1 1 1 ||| |||
! ) и регулирующим органам , ||| ! ) and in regulators , ||| 1 1.19187e-07 1 0.00118438 ||| 0-0 1-1 2-2 3-3 4-4 5-5 ||| 1 1 1 ||| |||
! ) и регулирующим органам , которые ||| ! ) and in regulators , who ||| 1 2.3692e-08 1 0.00017379 ||| 0-0 1-1 2-2 3-3 4-4 5-5 6-6 ||| 1 1 1 ||| |||
! ) и регулирующим органам ||| ! ) and in regulators ||| 1 1.60583e-07 1 0.00239106 ||| 0-0 1-1 2-2 3-3 4-4 ||| 1 1 1 ||| |||
! ) людям ||| ! ) to people ||| 1 0.0335482 0.5 0.00886362 ||| 0-0 1-1 2-3 ||| 1 2 1 ||| |||
! ) людям ||| listen ! ) to people ||| 1 0.0335482 0.5 1.08136e-07 ||| 0-1 1-2 2-4 ||| 1 2 1 ||| |||
! ) людям о ||| ! ) to people about ||| 0.5 0.00783575 0.5 0.0012305 ||| 0-0 1-1 2-3 3-4 ||| 2 2 1 ||| |||
! ) людям о ||| listen ! ) to people about ||| 0.5 0.00783575 0.5 1.50121e-08 ||| 0-1 1-2 2-4 3-5 ||| 2 2 1 ||| |||
! ) людям о существующих ||| ! ) to people about ||| 0.5 4.81115e-07 0.5

In [9]:
extracted = ''
with gzip.open('/mnt/DATA/MSU2024NLP/corpus/train/model/reordering-table.wbe-msd-bidirectional-fe.gz','rt') as f:
    for _ in range(10):
        extracted += next(f)
print(extracted)

! ! ! ||| ! ! ! ||| 0.6 0.2 0.2 0.6 0.2 0.2
! ! ||| ! ! ||| 0.714286 0.142857 0.142857 0.714286 0.142857 0.142857
! ! ||| ! ||| 0.6 0.2 0.2 0.6 0.2 0.2
! &amp; quot ; ||| ! ” ||| 0.714286 0.142857 0.142857 0.714286 0.142857 0.142857
! &amp; quot ; – ||| ! ” is ||| 0.6 0.2 0.2 0.6 0.2 0.2
! &amp; quot ; – требование , ||| ! ” is a demand ||| 0.6 0.2 0.2 0.6 0.2 0.2
! &amp; quot ; – требование ||| ! ” is a demand ||| 0.6 0.2 0.2 0.2 0.2 0.6
! &amp; quot ||| ! ” ||| 0.6 0.2 0.2 0.2 0.2 0.6
! &quot; , ||| ! &quot; ||| 0.6 0.2 0.2 0.6 0.2 0.2
! &quot; , а ||| ! &quot; rather ||| 0.6 0.2 0.2 0.6 0.2 0.2



## Подготовка тестовой и валидационной выборки

### Загрузка данных

Полученные данные получить в `/mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.<en/ru>`

In [63]:
# !wget https://object.pouta.csc.fi/OPUS-Books/v1/moses/en-ru.txt.zip
# !unzip unzip en-ru.txt.zip

### Готовим тестовые и валидационные данные по аналогии с обучающимися данными

In [64]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/tokenizer/tokenizer.perl \
        -l en \
        < /corpus/testing/Books.en-ru.en \
        > /corpus/testing/Books.en-ru.tok.en"

Tokenizer Version 1.1
Language: en
Number of threads: 1


In [65]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/tokenizer/tokenizer.perl \
        -l ru \
        < /corpus/testing/Books.en-ru.ru \
        > /corpus/testing/Books.en-ru.tok.ru"

Tokenizer Version 1.1
Language: ru
Number of threads: 1


In [66]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/recaser/truecase.perl \
        --model /corpus/truecase-model.en \
        < /corpus/testing/Books.en-ru.tok.en \
        > /corpus/testing/Books.en-ru.true.en"

In [67]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/recaser/truecase.perl \
        --model /corpus/truecase-model.ru \
        < /corpus/testing/Books.en-ru.tok.ru \
        > /corpus/testing/Books.en-ru.true.ru"

In [68]:
!wc -l /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.true.en

17496 /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.true.en


In [69]:
!wc -l /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.true.ru

17496 /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.true.ru


### Делим на тестовую часть и на валидационную (нужен небольшой объем данных)

In [70]:
!head -n 1000 /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.true.en > /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.val.true.en
!head -n 1000 /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.true.ru > /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.val.true.ru

!tail -n 1000 /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.true.en > /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.test.true.en
!tail -n 1000 /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.true.ru > /mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.test.true.ru

In [ ]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/bin/moses \
        -f /corpus/train/model/moses.ini \
        -threads 4 -s 40 -dl 8 \
        < /corpus/testing/Books.en-ru.test.true.ru \
        > /corpus/testing/Books.en-ru.test.true.en.TRANSLATED"

In [30]:
with open('/mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.test.true.ru', 'r') as fin:
    with open('/mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.test.true.en.TRANSLATED', 'r') as fout:
        for _ in range(5):
            text = MosesDetokenizer('ru').detokenize(next(fin).strip().split())  + '\n' \
                 + MosesDetokenizer('en').detokenize(next(fout).strip().split()) + '\n'
            print(text)

отчего же не потушить свечу, когда смотреть больше не на что, когда гадко смотреть на все это?
why is not calm свечу when look more than that, when гадко look at all this?

но как?
but how?

зачем этот кондуктор пробежал по жердочке, зачем они кричат, эти молодые люди в том вагоне?
why this кондуктор пробежал on жердочке, why should they кричат, these young people is вагоне?

зачем они говорят, зачем они смеются?
why should they say, why should they sneer?

все неправда, все ложь, все обман, все зло!.. "
it is untrue, all lies, all fraud, all evil!. "



In [73]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/generic/multi-bleu.perl \
        -lc /corpus/testing/Books.en-ru.test.true.en \
        < /corpus/testing/Books.en-ru.test.true.en.TRANSLATED"

It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 7.83, 47.7/14.5/5.5/2.1 (BP=0.831, ratio=0.844, hyp_len=20892, ref_len=24754)


## Подбор гиперпараметров на валидации

In [76]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/training/mert-moses.pl \
        --working-dir /corpus/train/mert-dir/ \
        --threads 4 \
        /corpus/testing/Books.en-ru.val.true.ru \
        /corpus/testing/Books.en-ru.val.true.en \
        /opt/moses/bin/moses /corpus/train/model/moses.ini \
        --mertdir /opt/moses/bin/ &> /corpus/mert.out"

In [77]:
!cat /mnt/DATA/MSU2024NLP/corpus/train/mert-dir/moses.ini

# MERT optimized configuration
# decoder /opt/moses/bin/moses
# BLEU 0.0654292 on dev /corpus/testing/Books.en-ru.val.true.ru
# We were before running iteration 26
# finished Thu Sep 26 23:19:46 UTC 2024
### MOSES CONFIG FILE ###
#########################

# input factors
[input-factors]
0

# mapping steps
[mapping]
0 T 0

[distortion-limit]
6

# feature functions
[feature]
UnknownWordPenalty
WordPenalty
PhrasePenalty
PhraseDictionaryMemory name=TranslationModel0 num-features=4 path=/corpus/train/model/phrase-table.gz input-factor=0 output-factor=0
LexicalReordering name=LexicalReordering0 num-features=6 type=wbe-msd-bidirectional-fe-allff input-factor=0 output-factor=0 path=/corpus/train/model/reordering-table.wbe-msd-bidirectional-fe.gz
Distortion
KENLM name=LM0 factor=0 path=/corpus/news-commentary-v8.ru-en.blm.en order=3

# dense weights for feature functions
[weight]

LexicalReordering0= 0.00137749 -0.0615252 0.122087 0.0459385 0.146033 0.162246
Distortion0= -0.0214977
LM0= 0.0788

In [ ]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/bin/moses \
        -f /corpus/train/mert-dir/moses.ini \
        -threads 4 -s 40 -dl 8 \
        < /corpus/testing/Books.en-ru.test.true.ru \
        > /corpus/testing/Books.en-ru.test.true.en.TRANSLATED.MERT"

In [31]:
with open('/mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.test.true.ru', 'r') as fin:
    with open('/mnt/DATA/MSU2024NLP/corpus/testing/Books.en-ru.test.true.en.TRANSLATED.MERT', 'r') as fout:
        for _ in range(5):
            text = MosesDetokenizer('ru').detokenize(next(fin).strip().split())  + '\n' \
                 + MosesDetokenizer('en').detokenize(next(fout).strip().split()) + '\n'
            print(text)

отчего же не потушить свечу, когда смотреть больше не на что, когда гадко смотреть на все это?
why is no longer on свечу calm that, when you look at the same time, when гадко look at all this?

но как?
but how?

зачем этот кондуктор пробежал по жердочке, зачем они кричат, эти молодые люди в том вагоне?
why this кондуктор кричат пробежал жердочке on these young people, why they вагоне in?

зачем они говорят, зачем они смеются?
why do they say, why should they sneer?

все неправда, все ложь, все обман, все зло!.. "
still, it is untrue, all of the lie, all of deception. "all the many evils!



In [2]:
!docker run -v /mnt/DATA/MSU2024NLP/corpus:/corpus haukurp/moses-smt:1.1.0 /bin/bash -c "\
        /opt/moses/scripts/generic/multi-bleu.perl \
        -lc /corpus/testing/Books.en-ru.test.true.en \
        < /corpus/testing/Books.en-ru.test.true.en.TRANSLATED.MERT"

It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 6.87, 41.6/11.6/3.7/1.3 (BP=1.000, ratio=1.065, hyp_len=26361, ref_len=24754)
